In [23]:

from decimal import getcontext
from decimal import Decimal


class SourceStates: # контейнер, содержащий алфавит сообщения для кодирования (состояния источника и вероятности их появления)
    @staticmethod
    def create_dict(message: str) -> dict: # статический метод для создания словаря [символ, кол-во появлений]
        states = {}
        for char in message:
            if char not in states:
                states[char] = 1
            else:
                states[char] += 1
        return states
    @staticmethod
    def create_freq_dict(states: dict, message: str) -> dict: # статический метод для создания словаря [символ, вероятность появления символа]
        freq_dict = {}
        for char, count in states.items():
            freq_dict[char] = count / len(message)
        return freq_dict
    @staticmethod
    def create_left_borders_dict(freq_dict: dict) -> dict: # левые границы кодирования (накопительные вероятности)
        left_borders_dict = {}
        sum = 0.0
        for char, freq in freq_dict.items():
            left_borders_dict[char] = sum
            sum += freq
        return left_borders_dict
    def __init__(self, message):
        self.states = self.create_dict(message)
        self.freq_states = self.create_freq_dict(self.states, message)
        self.left_borders_dict = self.create_left_borders_dict(self.freq_states)

getcontext().prec = 5000

class ArithmeticEncode:
    def __init__(self, message: str):
        self.message = message
        self.left = Decimal(0.0)
        self.right = Decimal(1.0)
        self.source_states = SourceStates(self.message)
        
    def encode(self) -> float:
        for char in self.message:
            left_border = Decimal(self.source_states.left_borders_dict[char])
            right_border = Decimal(self.source_states.left_borders_dict[char] + self.source_states.freq_states[char])
            current_range = self.right - self.left
            self.right = self.left + current_range * right_border
            self.left = self.left + current_range * left_border
        
        return (self.left + self.right) / 2
class ArithmeticDecode:
    def __init__(self, coded_val: float, message_len: int, source_states: SourceStates):
        self.coded_val = Decimal(coded_val)
        self.message_len = message_len
        self.source_states = source_states
        self.right = Decimal(1.0)
        self.left = Decimal(0.0)
        
    def decode(self) -> str:
        decoded_message = ''
        left_borders = self.source_states.left_borders_dict
        freq_dict = self.source_states.freq_states
        for i in range(self.message_len):
            current_range = self.right - self.left
            if current_range == 0: #выходим из цикла на границе точности чтобы избежать деления на ноль
                break
            scaled_value = (self.coded_val - self.left) / current_range
            for char, add_left_freq in left_borders.items():
                add_right_freq = add_left_freq + freq_dict[char]
                if add_left_freq <= scaled_value < add_right_freq:
                    decoded_message += char
                    self.right = self.left + current_range * Decimal(add_right_freq)
                    self.left = self.left + current_range * Decimal(add_left_freq)
        return decoded_message

file = open(r"..\cache\dz\ex1.txt", "r", encoding="utf-8")
data = file.read()
file.close()

code = ArithmeticEncode(data)
decode = ArithmeticDecode(code.encode(), len(data), SourceStates(data))
print(f"Encoded: {code.encode()}")
print(f"Decoded: {decode.decode()}")


Encoded: 0.00003565809477987437145557052239836428046328876742844138039561546467908097704572035260255021568752169031035886801992346866363617805282797284542507897769725218359882582300873391440071860507695690852743508728883638368358285798222404884788042568231674892050359953930638777470080722443481254365467986521606110808789115668291526577915508642105377412795363674495552327906177243330960523125619102384612419444830090411254773718806070305882441786068759922272319361422987009242178615829622373732578411109042187118236336672470815982097832966336045034955881320640231640714492378682558286218995826089596710890947005619365380984507484418630892687555029588498741414735555525247550424890333028693914533117797805027838058321434776044539981645938607553312205512749295409608950644249692978813534025574275287369354411729107578443709537599098507980964739317478173742525833361931412736373550498745400107673242772953661191589950689946156776615484810871975819512375408029586616780735972882961784683247027156237158

In [24]:
from fractions import Fraction
from decimal import getcontext
from decimal import Decimal

class SourceStates:  # контейнер, содержащий алфавит сообщения для кодирования (состояния источника и вероятности их появления)
    @staticmethod
    def create_dict(message: str) -> dict:  # статический метод для создания словаря [символ, кол-во появлений]
        states = {}
        for char in message:
            if char not in states:
                states[char] = 1
            else:
                states[char] += 1
        return states

    @staticmethod
    def create_freq_dict(states: dict, message: str) -> dict:  # статический метод для создания словаря [символ, вероятность появления символа]
        freq_dict = {}
        total = len(message)
        for char, count in states.items():
            freq_dict[char] = Fraction(count, total)  #используем дроби для вероятностей
        return freq_dict

    @staticmethod
    def create_left_borders_dict(freq_dict: dict) -> dict:  # левые границы кодирования (накопительные вероятности)
        left_borders_dict = {}
        sum_fraction = Fraction(0, 1)  #инициализация дроби 0/1
        for char, freq in freq_dict.items():
            left_borders_dict[char] = sum_fraction
            sum_fraction += freq
        return left_borders_dict

    def __init__(self, message):
        self.states = self.create_dict(message)
        self.freq_states = self.create_freq_dict(self.states, message)
        self.left_borders_dict = self.create_left_borders_dict(self.freq_states)

class ArithmeticEncode:
    def __init__(self, message: str):
        self.message = message
        self.left = Fraction(0, 1)  #левая граница 0/1 = 0
        self.right = Fraction(1, 1)  #правая граница 1/1 = 1
        self.source_states = SourceStates(self.message)

    def encode(self) -> Fraction:
        for char in self.message:
            left_border = self.source_states.left_borders_dict[char]
            right_border = left_border + self.source_states.freq_states[char]
            current_range = self.right - self.left
            self.right = self.left + current_range * right_border
            self.left = self.left + current_range * left_border

        return (self.left + self.right) / 2  #возвращает результат дробью

class ArithmeticDecode:
    def __init__(self, coded_val: Fraction, message_len: int, source_states: SourceStates):
        self.coded_val = coded_val
        self.message_len = message_len
        self.source_states = source_states
        self.right = Fraction(1, 1)  #левая граница 0/1 = 0
        self.left = Fraction(0, 1)  #правая граница 1/1 = 1

    def decode(self) -> str:
        decoded_message = ''
        left_borders = self.source_states.left_borders_dict
        freq_dict = self.source_states.freq_states
        for i in range(self.message_len):
            current_range = self.right - self.left
            if current_range == 0: 
                break
            scaled_value = (self.coded_val - self.left) / current_range
            for char, add_left_freq in left_borders.items():
                add_right_freq = add_left_freq + freq_dict[char]
                if add_left_freq <= scaled_value < add_right_freq:
                    decoded_message += char
                    self.right = self.left + current_range * add_right_freq
                    self.left = self.left + current_range * add_left_freq
        return decoded_message

# Usage
file = open(r"..\cache\dz\ex1.txt", "r", encoding="utf-8")
data = file.read()
file.close()

code = ArithmeticEncode(data)
encoded_value = code.encode()
decode = ArithmeticDecode(encoded_value, len(data), SourceStates(data))
print(f"Encoded: {encoded_value}")
print(f"Decoded: {decode.decode()}")

Encoded: 27169481178083648933606634944881599161436829055849047125979916490665517679513624102852151984336199875360856669573527456862459171209211303042680167365940585078560959683470508707038612203683456835752967170882643488419090786633313887089664821012406185207903972303592893886578587117235704145995558274008025693770684251682360920275923291469541/761944275088366696077898963040388548037992758779225338422551668483652185651557039908396435438442291395489784540538548978162721225576633899066101257563994550328087229163165445901894459650015485929538384238920499373351192191324992341637829290755508656940211713984325927880012097471573023765856191857892554253339767456054687500000000000000000000
Decoded: Все символы такого алфавита пронумерованы от 0 до 255, а каждому номеру соответствует 8-разрядный двоичный код от 00000000 до 11111111. Этот код является порядковым номером символа в двоичной системе счисления.
